# Topic Modeling

## NMF Topic Modeling

We will import a csv 'npr.csv' which contains news articles. Then we will try to apply topic modeling for the news articles using the NMF (Non-Negative Matrix Factorization).

The data has been obtained from NPR (National Public Radio) www.npr.org.

In [1]:
import pandas as pd

In [3]:
npr=pd.read_csv('npr.csv')

In [4]:
npr.head()

,Article
0,"In the Washington of 2016, even when the polic..."
1,Donald Trump has used Twitter — his prefe...
2,Donald Trump is unabashedly praising Russian...
3,"Updated at 2:50 p. m. ET, Russian President Vl..."
4,"From photography, illustration and video, to d..."


In [5]:
len(npr)

11992

So we have a total of 11992 documents in the dataframe.

### Preprocessing

Let's preprocess the articles using TfidfVectorizer from sklearn.

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [7]:
tfidf=TfidfVectorizer(max_df=0.95, min_df=2, stop_words='english')

In [8]:
dtm=tfidf.fit_transform(npr['Article'])

This step will return a term matrix for all the terms for all documents. Hence the size of the matrix is 11992x54777 which is number of documentsxwords.

In [9]:
dtm

<11992x54777 sparse matrix of type '<class 'numpy.float64'>'
	with 3033388 stored elements in Compressed Sparse Row format>

In [11]:
len(tfidf.get_feature_names())

54777

The above number shows us the different words/feature names present in cv.
We can check the feature name for any location from these 54777 features.

In [12]:
print(tfidf.get_feature_names()[0])
print(tfidf.get_feature_names()[8000])
print(tfidf.get_feature_names()[40000])
print(tfidf.get_feature_names()[54776])

00
canopy
receiving
überfunky


Now that we have the term matrix (dtm), we can use NMF model to fit the matrix.

In [13]:
from sklearn.decomposition import NMF

In [14]:
nmf_model=NMF(n_components=10, random_state=42)

In [15]:
nmf_model.fit(dtm)

NMF(n_components=10, random_state=42)

### Showing Top Words Per Topic

In [17]:
len(nmf_model.components_)

10

In [18]:
nmf_model.components_.shape

(10, 54777)

In [19]:
nmf_model.components_

array([[0.00076005, 0.26692258, 0.        , ..., 0.00266869, 0.00035543,
        0.        ],
       [0.        , 0.00066787, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.09803908, 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.03090463, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.00429197, 0.04813308, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.00445153, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

The above shows probability of every feature_name in tfidf per topic. Hence, the shape is 10x54777

Let's check the first feature_names for a single topic.

In [21]:
single_topic=nmf_model.components_[0]
single_topic

array([0.00076005, 0.26692258, 0.        , ..., 0.00266869, 0.00035543,
       0.        ])

In [22]:
# Returns the indices that would sort this array.
single_topic.argsort()

array([27388, 27031, 27030, ..., 19307, 36283, 42993], dtype=int64)

In [23]:
# Top 10 words for this topic:
single_topic.argsort()[-10:]

array([26752, 10425, 47218, 33390, 36310, 28659, 53152, 19307, 36283,
       42993], dtype=int64)

In [25]:
top_word_indices = single_topic.argsort()[-10:]
for index in top_word_indices:
    print(tfidf.get_feature_names()[index])

just
company
study
new
percent
like
water
food
people
says


Let's view the Top15 words for all the 10 topics.

In [26]:
for index,topic in enumerate(nmf_model.components_):
    print(f'THE TOP 15 WORDS FOR TOPIC #{index}')
    print([tfidf.get_feature_names()[i] for i in topic.argsort()[-15:]])
    print('\n')

THE TOP 15 WORDS FOR TOPIC #0
['year', 'university', 'workers', '000', 'years', 'just', 'company', 'study', 'new', 'percent', 'like', 'water', 'food', 'people', 'says']


THE TOP 15 WORDS FOR TOPIC #1
['administration', 'cruz', 'election', 'pence', 'gop', 'presidential', 'obama', 'house', 'white', 'republican', 'donald', 'campaign', 'said', 'president', 'trump']


THE TOP 15 WORDS FOR TOPIC #2
['patients', 'repeal', 'law', 'act', 'republicans', 'tax', 'people', 'plan', 'affordable', 'obamacare', 'coverage', 'medicaid', 'insurance', 'care', 'health']


THE TOP 15 WORDS FOR TOPIC #3
['assad', 'iran', 'iraq', 'north', 'china', 'aleppo', 'war', 'korea', 'said', 'forces', 'russia', 'military', 'syrian', 'syria', 'isis']


THE TOP 15 WORDS FOR TOPIC #4
['cruz', 'election', 'primary', 'democrats', 'percent', 'party', 'vote', 'state', 'delegates', 'democratic', 'hillary', 'campaign', 'voters', 'sanders', 'clinton']


THE TOP 15 WORDS FOR TOPIC #5
['book', 'love', 'women', 'way', 'time', 'life'

### Attaching Discovered Topic Labels to Original Articles

In [27]:
topic_results = nmf_model.transform(dtm)

In [28]:
topic_results.shape

(11992, 10)

In [29]:
topic_results[0].round(2)

array([0.  , 0.1 , 0.  , 0.06, 0.01, 0.  , 0.  , 0.  , 0.  , 0.08])

In [30]:
topic_results[0].argmax()

1

This means that our model thinks that the first article belongs to topic #1

### Combining with Original Data

In [31]:
npr.head()

,Article
0,"In the Washington of 2016, even when the polic..."
1,Donald Trump has used Twitter — his prefe...
2,Donald Trump is unabashedly praising Russian...
3,"Updated at 2:50 p. m. ET, Russian President Vl..."
4,"From photography, illustration and video, to d..."


In [32]:
topic_results.argmax(axis=1)

array([1, 1, 1, ..., 7, 4, 0], dtype=int64)

In [33]:
npr['Topic'] = topic_results.argmax(axis=1)

In [34]:
npr.head(10)

,Article,Topic
0,"In the Washington of 2016, even when the polic...",1
1,Donald Trump has used Twitter — his prefe...,1
2,Donald Trump is unabashedly praising Russian...,1
3,"Updated at 2:50 p. m. ET, Russian President Vl...",3
4,"From photography, illustration and video, to d...",6
5,I did not want to join yoga class. I hated tho...,5
6,With a who has publicly supported the debunk...,0
7,"I was standing by the airport exit, debating w...",0
8,"If movies were trying to be more realistic, pe...",0
9,"Eighteen years ago, on New Year’s Eve, David F...",0


Here, we have added the Topic column in the npr dataframe, where Topic number resembles one of the 10 topics calculated by NMF.

------------------------------------------------------------------------------------------------------------------------------

### Mapping topic to the documents
Here, first try to manually assign a topic name to the 10 topics depending on the Top 15 words. Then we add the topic names to npr dataframe.

In [35]:
topic_name={0:'Education',1:'Elections',2:'Life Insurance',3:'Politics',4:'Election',5:'Music',6:'Education',7:'Healthcare',8:'Law&Order',9:'Politics'}

In [37]:
npr['topic_name']=npr['Topic'].map(topic_name)

In [38]:
npr.head()

,Article,Topic,topic_name
0,"In the Washington of 2016, even when the polic...",1,Elections
1,Donald Trump has used Twitter — his prefe...,1,Elections
2,Donald Trump is unabashedly praising Russian...,1,Elections
3,"Updated at 2:50 p. m. ET, Russian President Vl...",3,Politics
4,"From photography, illustration and video, to d...",6,Education
